In [1]:
from functools import reduce
import csbgnpy.pd.io.sbgnml
import pypint
import multiprocessing
from IPython.display import display
import networkx as nx
from tqdm.notebook import tqdm

This notebook has been executed using the docker image `pauleve/clockcycle:v1`

## Merged map

From [1 - Merge CYCLE and CLOCK](1%20-%20Merge%20CYCLE%20and%20CLOCK.ipynb)

In [2]:
merge_file = "generated/MERGE.sbgnml"
merge = csbgnpy.pd.io.sbgnml.read(merge_file)

In [3]:
clock = csbgnpy.pd.io.sbgnml.read("generated/CLOCK.sbgnml")
cycle = csbgnpy.pd.io.sbgnml.read("generated/CYCLE.sbgnml")
clock_ids = set([e.id for e in clock.entities])# + [p.id for p in clock.processes])
cycle_ids = set([e.id for e in cycle.entities])# + [p.id for p in cycle.processes])
clock_only = clock_ids - cycle_ids
cycle_only = cycle_ids - clock_ids

## Markers

From [2 - Markers.ipynb](2%20-%20Markers.ipynb):

In [4]:
%store -r cycle_markers
%store -r clock_markers

## Initial state

From [3 - Initial state](3%20-%20Initial%20state.ipynb)

In [5]:
%store -r inis

To ids:

In [6]:
inis = {code: [e.id for e in ini] for code, ini in inis.items()}

### Conversion to internal identifiers

In [7]:
def get_map(a):
    if a in clock_only:
        return "CLOCK"
    if a in cycle_only:
        return "CYCLE"
    return "MERGE"
def pretty_node(a):
    if a.startswith("epn_"):
        e = merge.get_entity(a, by_id=True)
    else:
        e = merge.get_process(a, by_id=True)
    return "{}::{}".format(get_map(a), str(e))
def get_entity_id(spec):
    e = merge.get_entity(spec, by_string=True)
    if not e:
        raise Exception(f"No entity found for '{spec}'")
    return e.id

In [8]:
an_cycle_markers = dict([(p, [(get_entity_id(m),1) for m in ms]) \
                         for p,ms in cycle_markers.items()])
an_clock_markers = dict([(p, [(get_entity_id(m),1) for m in ms]) \
                         for p,ms in clock_markers.items()])

In [9]:
def pretty_mutations(m):
    return dict([(pretty_node(a), i) for (a,i) in sorted(m.items())])

def has_proc(m):
    for e in m:
        if e.startswith("proc_"):
            return True
    return False

def unpretty_node(a):
    name = "::".join(a.split("::")[1:])
    entity = merge.get_entity(name, by_string=True)
    return entity.id

def unpretty_mutations(m):
    return dict([(unpretty_node(a), i) for (a,i) in m.items()])

In [10]:
def oneshot_mutations_for_cut(an, markers, **kwargs):
    ms = []
    for marker in markers:
        ms += an.oneshot_mutations_for_cut(marker, **kwargs)
    return ms

In [11]:
def check_mutation_for_cut(an, markers, locks):
    for marker in markers:
        wt_reach = an.reachability(marker)
        mu_reach = an.lock(locks).reachability(marker)
        if wt_reach and not mu_reach:
            print("mutation is cutting {}".format(pretty_node(marker[0])))

In [12]:
def list_interesting(ms, interest):
    i2m = {}
    for mi, m in enumerate(ms):
        li = [(a,i) for (a,i) in m.items() if a in interest]
        arity = len(m)
        for (a,i) in li:
            a = pretty_node(a)
            if (a,i) not in i2m:
                i2m[(a,i)] = set()
            i2m[(a,i)].add(arity)
            #if arity not in i2m[(a,i)]:
            #    i2m[(a,i)][arity] = []
            #i2m[(a,i)][arity].append(mi)
    si2m = {}
    for k, v in sorted(i2m.items()):
        si2m[k] = v
    return si2m

In [13]:
ans = {code: pypint.load(merge_file, initial_state=ini)
           for code, ini in tqdm(inis.items())}

In [14]:
def list_cutsets(an, markers, scope, **opts):
    ms = an.cutsets(markers, quiet=True, **opts)
    return frozenset(list_interesting(ms, scope).keys())

## CYCLE cut sets

In [15]:
cuts = {}
for phase in tqdm(an_cycle_markers):
    cuts[phase] = {code: list_cutsets(an, an_cycle_markers[phase], clock_ids, 
                                      exclude_initial_state=False)
                        for code, an in tqdm(ans.items(), phase)}

In [16]:
{phase: len(set(r.values())) for phase, r in cuts.items()}

{'earlyG1': 1, 'lateG1': 1, 'earlyS': 1, 'lateS': 1, 'G2': 3, 'M': 5}

In [17]:
cycle_cuts = cuts
%store cycle_cuts

Stored 'cycle_cuts' (dict)


In [18]:
def summary(spec):
    (a,i) = spec
    name = a.split("#")[0].split('(')[-1].split(']')[-1]
    mut = "absent" if i == 0 else "present"
    return (name, mut)

def reindex(results):
    imutations = {}
    for phase, mut in results.items():
        imutations[phase] = {}
        for code, value in mut.items():
            value = frozenset([summary(m) for m in value])
            if value not in imutations[phase]:
                imutations[phase][value] = {code}
            else:
                imutations[phase][value].add(code)
    return imutations

In [19]:
reindex(cycle_cuts)

{'earlyG1': {frozenset({('HSP90', 'present')}): {'1111',
   '1112',
   '1121',
   '1122',
   '1211',
   '1212',
   '1221',
   '1222',
   '2111',
   '2112',
   '2121',
   '2122',
   '2211',
   '2212',
   '2221',
   '2222',
   '3111',
   '3112',
   '3121',
   '3122',
   '3211',
   '3212',
   '3221',
   '3222'}},
 'lateG1': {frozenset({('HSP90', 'present')}): {'1111',
   '1112',
   '1121',
   '1122',
   '1211',
   '1212',
   '1221',
   '1222',
   '2111',
   '2112',
   '2121',
   '2122',
   '2211',
   '2212',
   '2221',
   '2222',
   '3111',
   '3112',
   '3121',
   '3122',
   '3211',
   '3212',
   '3221',
   '3222'}},
 'earlyS': {frozenset(): {'1111',
   '1112',
   '1121',
   '1122',
   '1211',
   '1212',
   '1221',
   '1222',
   '2111',
   '2112',
   '2121',
   '2122',
   '2211',
   '2212',
   '2221',
   '2222',
   '3111',
   '3112',
   '3121',
   '3122',
   '3211',
   '3212',
   '3221',
   '3222'}},
 'lateS': {frozenset({('HSP90', 'present')}): {'1111',
   '1112',
   '1121',
   '1122',


## CLOCK markers

In [20]:
cuts = {}
for phase in tqdm(an_clock_markers):
    cuts[phase] = {code: list_cutsets(an, an_clock_markers[phase], cycle_ids, 
                                      exclude_initial_state=False)
                        for code, an in tqdm(ans.items(), phase)}

In [21]:
{phase: len(set(r.values())) for phase, r in cuts.items()}

{'RORG': 3, 'SIRT1': 1, 'ARNTL-CLOCK': 3, 'PER-CRY': 3}

In [22]:
clock_cuts = cuts
%store clock_cuts

Stored 'clock_cuts' (dict)


In [23]:
reindex(clock_cuts)

{'RORG': {frozenset({('TP53', 'present')}): {'1111',
   '1121',
   '1211',
   '1221',
   '3111',
   '3121',
   '3211',
   '3221'},
  frozenset({('MDM2', 'present'), ('TP53', 'present')}): {'1112',
   '1122',
   '1212',
   '1222',
   '3112',
   '3122',
   '3212',
   '3222'},
  frozenset(): {'2111',
   '2112',
   '2121',
   '2122',
   '2211',
   '2212',
   '2221',
   '2222'}},
 'SIRT1': {frozenset(): {'1111',
   '1112',
   '1121',
   '1122',
   '1211',
   '1212',
   '1221',
   '1222',
   '2111',
   '2112',
   '2121',
   '2122',
   '2211',
   '2212',
   '2221',
   '2222',
   '3111',
   '3112',
   '3121',
   '3122',
   '3211',
   '3212',
   '3221',
   '3222'}},
 'ARNTL-CLOCK': {frozenset({('TP53', 'present')}): {'1111',
   '1121',
   '1211',
   '1221',
   '3111',
   '3121',
   '3211',
   '3221'},
  frozenset({('MDM2', 'present'), ('TP53', 'present')}): {'1112',
   '1122',
   '1212',
   '1222',
   '3112',
   '3122',
   '3212',
   '3222'},
  frozenset(): {'2111',
   '2112',
   '2121',
   '21